# Data Exploration

In [8]:
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
import pickle

In [9]:
# Display Options for Pandas
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = None

In [10]:
data_path = Path("../data")
data_path_string = data_path.resolve().as_posix()

### ETL and EDA

In [83]:
con = duckdb.connect(database=f"{data_path}/spotify.db")

In [12]:
def clean_db(con: duckdb.DuckDBPyConnection):
    con.execute("drop table if exists albums")
    con.execute("drop table if exists artists")
    con.execute("drop table if exists tracks")
    con.execute("drop table if exists features")
    con.execute("drop table if exists genres")
    con.execute("drop table if exists features_preprocessed")

clean_db(con)

In [13]:
def load_data_into_db(con: duckdb.DuckDBPyConnection, data_path_string: str):
    con.read_csv(f"{data_path_string}/spotify_albums.csv", all_varchar=False).create("albums")
    con.read_csv(f"{data_path_string}/spotify_artists.csv", all_varchar=False).create("artists")
    con.read_csv(f"{data_path_string}/spotify_tracks.csv", all_varchar=False).create("tracks")

load_data_into_db(con, data_path_string)

In [14]:
con.table("albums").limit(1).df()

,column00,album_type,artist_id,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,track_id,track_name_prev,uri,type
0,0,single,3DiDSECUqqY1AuBP8qtaIa,"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",{'spotify': 'https://open.spotify.com/album/1gAM7M4rBwEbSPeAQR2nx1'},https://api.spotify.com/v1/albums/1gAM7M4rBwEbSPeAQR2nx1,1gAM7M4rBwEbSPeAQR2nx1,"[{'height': 640, 'url': 'https://i.scdn.co/image/5872e4d8fac4ef7552576d481b1d676189b4056a', 'wid...",If I Ain't Got You EP,2019-02-08,day,6,2iejTMy9XZ8Gaae0aQ2yl0,track_32,spotify:album:1gAM7M4rBwEbSPeAQR2nx1,album


In [15]:
con.table("artists").limit(3).df()

,column0,artist_popularity,followers,genres,id,name,track_id,track_name_prev,type
0,0,44,23230,"['sertanejo', 'sertanejo pop', 'sertanejo tradicional', 'sertanejo universitario']",4mGnpjhqgx4RUdsIJiURdo,Juliano Cezar,0wmDmAILuW9e2aRttkl4aC,track_9,artist
1,1,22,313,[],1dLnVku4VQUOLswwDFvRc9,The Grenadines,4wqwj0gA8qPZKLl5WVqXml,track_30,artist
2,2,26,1596,['danish pop rock'],6YVY310fjfUzKi8hiqR7iK,Gangway,1bFqWDbvHmZe2f4Nf9qaD8,track_38,artist


In [16]:
con.table("tracks").limit(3).df()

,column00,acousticness,album_id,analysis_url,artists_id,available_markets,country,danceability,disc_number,duration_ms,energy,href,id,instrumentalness,key,liveness,loudness,lyrics,mode,name,playlist,popularity,preview_url,speechiness,tempo,time_signature,track_href,track_name_prev,track_number,uri,valence,type
0,0,0.294,0D3QufeCudpQANOR7luqdr,https://api.spotify.com/v1/audio-analysis/5qljLQuKnNJf4F4vfxQB0V,['3mxJuHRn2ZWD5OofvJtDZY'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",BE,0.698,1.0,235584.0,0.606,https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V,5qljLQuKnNJf4F4vfxQB0V,0.000003,10.0,0.151,-7.447,"\r\n\r\nPerhaps I am bound to be restless\r\nAlways yearning, never satisfied\r\nPerhaps I'm ach...",0.0,Blood,Hipsteribrunssi,28.0,https://p.scdn.co/mp3-preview/1b05a902da3a251d07a38aa710ffae559fc33d08?cid=b3cdb16d0df2409abf6a8...,0.0262,115.018,4.0,https://api.spotify.com/v1/tracks/5qljLQuKnNJf4F4vfxQB0V,track_14,1.0,spotify:track:5qljLQuKnNJf4F4vfxQB0V,0.622,track
1,1,0.863,1bcqsH5UyTBzmh9YizdsBE,https://api.spotify.com/v1/audio-analysis/3VAX2MJdmdqARLSU5hPMpm,['4xWMewm6CYMstu0sPgd9jJ'],"['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY',...",BE,0.719,1.0,656960.0,0.308,https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm,3VAX2MJdmdqARLSU5hPMpm,0.000000,6.0,0.253,-10.340,\r\nYour Gods and my Gods-do you or I know which are the stronger? Native Proverb.\r\nEAST of Su...,1.0,The Ugly Duckling,Animal Stories,31.0,https://p.scdn.co/mp3-preview/d8140736a6131cb5595f061975173a272c343a0a?cid=b3cdb16d0df2409abf6a8...,0.9220,115.075,3.0,https://api.spotify.com/v1/tracks/3VAX2MJdmdqARLSU5hPMpm,track_3,3.0,spotify:track:3VAX2MJdmdqARLSU5hPMpm,0.589,track
2,2,0.750,4tKijjmxGClg4JOLAyo2qE,https://api.spotify.com/v1/audio-analysis/1L3YAhsEMrGVvCgDXj2TYn,['3hYaK5FF3YAglCj5HZgBnP'],['GB'],BE,0.466,1.0,492840.0,0.931,https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn,1L3YAhsEMrGVvCgDXj2TYn,0.000000,4.0,0.938,-13.605,"\r\n\r\nClosed off from love, I didn't need the pain\r\nOnce or twice was enough and it was all ...",0.0,Jimmy Launches His Own Range Of Greetings Cards,Best Of British Comedy,31.0,https://p.scdn.co/mp3-preview/c8af28fb15185b18977152eb50eefef8d90af5a2?cid=b3cdb16d0df2409abf6a8...,0.9440,79.565,4.0,https://api.spotify.com/v1/tracks/1L3YAhsEMrGVvCgDXj2TYn,track_4,4.0,spotify:track:1L3YAhsEMrGVvCgDXj2TYn,0.085,track


The tracks table contains much more tracks than the artists table. Joining both tables will lead to some losses.

In [105]:
con.query(""" 
    select 
        (select count(distinct track_id) from artists) as artists_track_id,
        (select count(distinct id) from tracks) as tracks_track_id,
        (select count(distinct id) from artists) as artists_artists_id,
        (select count(distinct artists_id) from tracks) as tracks_artists_id
""").df()

,artists_track_id,tracks_track_id,artists_artists_id,tracks_artists_id
0,44895,101939,56129,54673


In [108]:
# Construct Genre Table (with Polars because it's faster than pandas)
def create_genre_table(con: duckdb.DuckDBPyConnection):
    df = con.query("select track_id, genres from artists").pl()
    df = (
        # Explode genre array into rows for each genre
        df.with_columns(
            pl.col("genres").str.replace("^\[\]$", "NoGenre").str.replace_all("hip hop", "hiphop")
        ).with_columns(
            pl.col("genres").str.strip("[]").str.replace_all("'", "").str.split(", ")
        )
        .explode("genres")

        # Explode each genre into words
        .with_columns(
            pl.col("genres").str.split(" ").alias("words")
        )
        .explode("words")
    )

    # Get top X genre words by count (like "pop" or "rock")
    top_genre = (
        df
        .groupby("words")
        .agg([
            # word count is also the ranking
            pl.count("words").alias("count"),
        ])
        .filter(pl.col("words") != "NoGenre")
        .sort("count", descending=True)
        .limit(20)
    )

    # final genre df
    genre_final = (
        df
        .join(top_genre, on="words", how="left")
        .with_columns(
            pl.when(pl.col("count").is_null()).then("Other").otherwise(pl.col("words")).alias("genre_class")
        )
        .sort("count", descending=True)
        .unique(subset="track_id", keep="first")
    )
    
    # Back to DuckDB
    sql = """ 
        drop table if exists genres;
        create table genres as 
        select track_id, genres, genre_class from genre_final
    """
    con.execute(sql)

create_genre_table(con)

In [109]:
con.table("genres").limit(3).df()

,track_id,genres,genre_class
0,3KKlsrWi8jw6SaeYF0YcHO,shiver pop,pop
1,2xK48geuNoJAGW1tT073N6,pop rap,pop
2,4MKdubfwlQ0HHCHM07pMa9,christian pop,pop


We have suffered no losses of tracks from creating the genres table:

In [112]:
con.query("""
    select 
        (select count(distinct track_id) from genres) as genres_track_id,
        (select count(distinct track_id) from artists) as artists_track_id        
""").df()

,genres_track_id,artists_track_id
0,44895,44895


In [169]:
# Construct Features Table
def create_features_table(con: duckdb.DuckDBPyConnection):
    sql = """
        drop table if exists features;
        create table features as
        select
            t.id, 
            t.acousticness, t.danceability, t.energy, t.instrumentalness, t.liveness, t.loudness, t.speechiness, t.tempo, t.valence,
            t.name as track_name, ar.name as artist_name, a.name as album_name, 
            coalesce(g.genres, 'Other') as genres, 
            coalesce(g.genre_class, 'Other') as genre_class,
            t.preview_url, t.track_href, t.analysis_url,
            row_number() over (order by t.id) -1 as row_number
        from tracks t
        join albums a on t.album_id = a.id
        join artists ar on a.artist_id = ar.id
        left join genres g on t.id = g.track_id
        order by t.id
        """
    con.execute(sql)
    
create_features_table(con)

In [170]:
con.table("features").limit(3).df()

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_name,artist_name,album_name,genres,genre_class,preview_url,track_href,analysis_url,row_number
0,000RDCYioLteXcutOjeweY,0.05830,0.679,0.770,0.000000,0.0825,-3.537,0.1900,161.721,0.839,Teeje Week,Jordan Sandhu,Teeje Week,Other,Other,https://p.scdn.co/mp3-preview/d99a55fb91addb9fe3a8d1bdf6483be0c3422949?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/000RDCYioLteXcutOjeweY,https://api.spotify.com/v1/audio-analysis/000RDCYioLteXcutOjeweY,0
1,000YQJ9lmNX02OxJ7hEh4R,0.00926,0.397,0.964,0.000000,0.0776,-6.246,0.1700,169.679,0.330,Punanen risti,Karleby Fittans,Punanen risti,finnish punk,Other,https://p.scdn.co/mp3-preview/1e4866544a3c3ba0b23e27a85cb22591087960c1?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/000YQJ9lmNX02OxJ7hEh4R,https://api.spotify.com/v1/audio-analysis/000YQJ9lmNX02OxJ7hEh4R,1
2,001O0fWr1NyU700N1XzN5r,0.94000,0.360,0.164,0.000002,0.6700,-12.269,0.0471,79.309,0.205,Sig det Nu I ( Forgot),Various Artists,Toppen Af Poppen 2018 synger Turboweekend,classic danish pop,pop,https://p.scdn.co/mp3-preview/45cd6d7b0d1175f4d4da42264db00c9aabfe3163?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/001O0fWr1NyU700N1XzN5r,https://api.spotify.com/v1/audio-analysis/001O0fWr1NyU700N1XzN5r,2


In [171]:
con.query("summarize features").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,000RDCYioLteXcutOjeweY,7zzpno7uAqkAzWZDQuGEFA,100824,NaN,NaN,NaN,NaN,NaN,101144,0.0%
1,acousticness,DOUBLE,0.0,0.996,4628,0.3508251540038849,0.33448036356580496,0.040585283590515914,0.23679295352809757,0.6417199431894404,101144,0.0%
2,danceability,DOUBLE,0.0,0.989,1196,0.5860919144981468,0.17772637138480055,0.47976571008908586,0.6102875045289868,0.7144403187043746,101144,0.0%
3,energy,DOUBLE,0.0,1.0,2346,0.5873793629943553,0.25966898964390234,0.4131386707551002,0.6292959247316862,0.7979993149483935,101144,0.0%
4,instrumentalness,DOUBLE,0.0,1.0,5341,0.14856264550838277,0.3038408865949879,0.0,3.9147190071935604e-05,0.03551628415492234,101144,0.0%
5,liveness,DOUBLE,0.0,0.999,1715,0.1974682719686772,0.17525222623044312,0.09553627312127376,0.12369755008763077,0.2409240597503108,101144,0.0%
6,loudness,DOUBLE,-60.0,2.719,22234,-9.435085007514118,6.174813953268192,-11.115671355896463,-7.588933523669162,-5.503618130391811,101144,0.0%
7,speechiness,DOUBLE,0.0,0.969,1638,0.12792117278335627,0.20161193836671848,0.03636247338807088,0.050594656106883865,0.10370375863781452,101144,0.0%
8,tempo,DOUBLE,0.0,244.035,50033,118.39231111088999,30.21992223551469,95.90142111672085,118.33199776309228,136.21481320064538,101144,0.0%
9,valence,DOUBLE,0.0,0.993,1762,0.48269840187653157,0.26157282648219243,0.27084302125216636,0.4768566080893343,0.6937367469964331,101144,0.0%


In [172]:
# Count songs by genre word
sql = """ 
    select 
        genre_class, count(genre_class) as count
    from features
    group by genre_class
    order by count(genre_class) desc
"""
con.query(sql).df()

,genre_class,count
0,Other,79441
1,pop,7902
2,rock,3103
3,indie,2095
4,hiphop,1804
5,house,1379
6,classical,1277
7,jazz,975
8,metal,597
9,deep,324


## ML

In [173]:
def load_data_into_pd(con: duckdb.DuckDBPyConnection) -> pd.DataFrame:
    return con.query("select * from features").df()

df = load_data_into_pd(con)

In [174]:
df.columns

Index(['id', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'track_name',
       'artist_name', 'album_name', 'genres', 'genre_class', 'preview_url',
       'track_href', 'analysis_url', 'row_number'],
      dtype='object')

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101144 entries, 0 to 101143
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                101144 non-null  object 
 1   acousticness      101144 non-null  float64
 2   danceability      101144 non-null  float64
 3   energy            101144 non-null  float64
 4   instrumentalness  101144 non-null  float64
 5   liveness          101144 non-null  float64
 6   loudness          101144 non-null  float64
 7   speechiness       101144 non-null  float64
 8   tempo             101144 non-null  float64
 9   valence           101144 non-null  float64
 10  track_name        101144 non-null  object 
 11  artist_name       101144 non-null  object 
 12  album_name        101144 non-null  object 
 13  genres            101144 non-null  object 
 14  genre_class       101144 non-null  object 
 15  preview_url       101144 non-null  object 
 16  track_href        10

In [176]:
df.describe()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,row_number
count,101144.000000,101144.000000,101144.000000,101144.000000,101144.000000,101144.000000,101144.000000,101144.000000,101144.000000,101144.000000
mean,0.350825,0.586092,0.587379,0.148563,0.197468,-9.435085,0.127921,118.392311,0.482698,50571.500000
std,0.334480,0.177726,0.259669,0.303841,0.175252,6.174814,0.201612,30.219922,0.261573,29197.902151
min,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000
25%,0.040400,0.480000,0.413000,0.000000,0.095500,-11.112000,0.036400,95.983000,0.271000,25285.750000
50%,0.236000,0.610000,0.629000,0.000038,0.124000,-7.586000,0.050600,118.091000,0.477000,50571.500000
75%,0.642000,0.714250,0.798000,0.034200,0.241000,-5.504000,0.103000,136.053250,0.693000,75857.250000
max,0.996000,0.989000,1.000000,1.000000,0.999000,2.719000,0.969000,244.035000,0.993000,101143.000000


In [177]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

def create_features_processed_table(con: duckdb.DuckDBPyConnection):
    # Define X and y
    df = con.query("select * from features order by row_number").df()
    cols = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence']
    y = df["genre_class"]
    X = df[cols]

    # Scale X
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Encode y
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)

    # Concat DataFrames
    df_preprocessed = pd.concat([pd.DataFrame(X_scaled, columns=cols), pd.DataFrame(y_encoded, columns=["genre_class"])], axis=1)
    df_preprocessed = pd.concat([df[["id"]], df_preprocessed], axis=1)
  
    # Write to DuckDB
    con.execute("""
        drop table if exists features_preprocessed;
        create table features_preprocessed as
            select *, row_number() over (order by id) - 1 as row_number
            from df_preprocessed
            order by id
    """)

create_features_processed_table(con)

In [178]:
con.query("select * from features_preprocessed order by id").limit(5).df()

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre_class,row_number
0,000RDCYioLteXcutOjeweY,0.058534,0.686552,0.7700,0.000000,0.082583,0.900254,0.196078,0.662696,0.844914,0,0
1,000YQJ9lmNX02OxJ7hEh4R,0.009297,0.401416,0.9640,0.000000,0.077678,0.857061,0.175439,0.695306,0.332326,0,1
2,001O0fWr1NyU700N1XzN5r,0.943775,0.364004,0.1640,0.000002,0.670671,0.761029,0.048607,0.324990,0.206445,15,2
3,002F8HWG4UVMxby4Z2aOFI,0.902610,0.219414,0.0815,0.004730,0.138138,0.700171,0.039525,0.552552,0.075227,0,3
4,004JmuI6jYQMYJCvNpQW9z,0.523092,0.651163,0.2490,0.000000,0.427427,0.634513,0.993808,0.365456,0.413897,0,4


In [180]:
from sklearn.neighbors import KNeighborsClassifier

def get_model_knn() -> KNeighborsClassifier:
    # Check if model exists
    if Path(data_path / "knn.pkl").exists():
        with open(data_path / "knn.pkl", "rb") as f:
            return pickle.load(f)

    # Otherwise, create model
    df = con.query("select * from features_preprocessed order by id").df()
    X = df.drop(["id", "genre_class", "row_number"], axis=1)
    y = df["genre_class"]
    
    knn = KNeighborsClassifier(n_neighbors=5, algorithm="ball_tree")
    knn.fit(X,y)
    
    with open(data_path / "knn.pkl", "wb") as f:
        pickle.dump(knn, f)

    return knn

knn = get_model_knn()

In [181]:
# Search for a song
def lookup_song(con: duckdb.DuckDBPyConnection, lookup_query: str = "", genre_class: str = "") -> str:
    sql = """
      select *
      from features
      where 
        regexp_matches(lower(concat(track_name, ' ', artist_name)), $query)
        AND 
        regexp_matches(lower(genre_class), $genre)
      order by id
      limit 10
    """
    return con.execute(sql, { "query": lookup_query.lower(), "genre": genre_class.lower()}).fetch_df()

lookup_song(con, lookup_query="kid cudi", genre_class="")

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_name,artist_name,album_name,genres,genre_class,preview_url,track_href,analysis_url,row_number
0,5owJQbueZftSai4CuVlR9J,0.12300,0.767,0.722,0.003270,0.118,-5.562,0.0601,130.023,0.962,Day 'N' Nite - Radio Edit,Kid Cudi,Day 'N' Nite,fidget house,house,https://p.scdn.co/mp3-preview/3e575bca9959101a1ca1585315cd55186d2d2141?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/5owJQbueZftSai4CuVlR9J,https://api.spotify.com/v1/audio-analysis/5owJQbueZftSai4CuVlR9J,75606
1,5xYC48nOppVemY6U5GRGTb,0.00147,0.546,0.916,0.000004,0.251,-3.931,0.2550,129.986,0.380,Memories (feat. Kid Cudi),David Guetta,One More Love,pop rap,pop,https://p.scdn.co/mp3-preview/e505a03517400946e81bce7c57e16afac731f7f1?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/5xYC48nOppVemY6U5GRGTb,https://api.spotify.com/v1/audio-analysis/5xYC48nOppVemY6U5GRGTb,77336
2,7KPr0YxECy4Q1k2F17Sa0Q,0.00158,0.546,0.899,0.000006,0.229,-3.937,0.2330,130.003,0.470,Memories (feat. Kid Cudi),David Guetta,Memories (feat. Kid Cudi),Other,Other,https://p.scdn.co/mp3-preview/9364e08b3d1452c0afd107b6f21e9f9140450812?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/7KPr0YxECy4Q1k2F17Sa0Q,https://api.spotify.com/v1/audio-analysis/7KPr0YxECy4Q1k2F17Sa0Q,95103
3,7rn94fZKrOwHunAgWtvJUs,0.00150,0.544,0.870,0.000003,0.246,-6.103,0.2600,129.984,0.502,Memories (feat. Kid Cudi),David Guetta,One More Love,Other,Other,https://p.scdn.co/mp3-preview/7798e1790d4e1b791fb2615b7bcce4fac655deb1?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/7rn94fZKrOwHunAgWtvJUs,https://api.spotify.com/v1/audio-analysis/7rn94fZKrOwHunAgWtvJUs,99409


In [182]:
# Show song details
def show_song_details(con: duckdb.DuckDBPyConnection, track_id: str) -> str:
    sql = """select * from features where id = $track_id"""
    return con.execute(sql, { "track_id": track_id}).fetch_df()


show_song_details(con, "5owJQbueZftSai4CuVlR9J")

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_name,artist_name,album_name,genres,genre_class,preview_url,track_href,analysis_url,row_number
0,5owJQbueZftSai4CuVlR9J,0.123,0.767,0.722,0.00327,0.118,-5.562,0.0601,130.023,0.962,Day 'N' Nite - Radio Edit,Kid Cudi,Day 'N' Nite,fidget house,house,https://p.scdn.co/mp3-preview/3e575bca9959101a1ca1585315cd55186d2d2141?cid=b3cdb16d0df2409abf6a8...,https://api.spotify.com/v1/tracks/5owJQbueZftSai4CuVlR9J,https://api.spotify.com/v1/audio-analysis/5owJQbueZftSai4CuVlR9J,75606


In [165]:
track_id = "5owJQbueZftSai4CuVlR9J"
sql = """select * from features_preprocessed where id = $track_id order by id"""
df_test = con.execute(sql, { "track_id": track_id}).fetch_df()
X_test = df_test.drop(["id", "genre_class", "row_number"], axis=1)
y_test = df_test["genre_class"]

knn = get_model_knn()
distances, indices = knn.kneighbors(X_test, n_neighbors=10)

In [166]:
df_test

,id,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre_class,row_number
0,5owJQbueZftSai4CuVlR9J,0.123494,0.775531,0.722,0.00327,0.118118,0.867967,0.062023,0.532805,0.968781,8,75606


In [188]:
distances

array([[0.        , 0.0464623 , 0.06727669, 0.07028612, 0.08008251,
        0.08011492, 0.08011649, 0.08288915, 0.08344958, 0.08349888]])

In [187]:
indices

array([[75606, 40105, 11698, 81650, 70548, 71566, 36521, 34201, 86600,
        20126]], dtype=int64)

In [189]:
recommended = pd.DataFrame({"row_number": indices[0], "distances": distances[0]})
recommended

,rows,distances
0,75606,0.000000
1,40105,0.046462
2,11698,0.067277
3,81650,0.070286
4,70548,0.080083
5,71566,0.080115
6,36521,0.080116
7,34201,0.082889
8,86600,0.083450
9,20126,0.083499


In [196]:
def make_recommendations(con: duckdb.DuckDBPyConnection, track_id: str, n_neighbors: int = 10) -> pd.DataFrame:
    # Get test data
    sql = """select * from features_preprocessed where id = $track_id order by id"""
    df_test = con.execute(sql, { "track_id": track_id}).fetch_df()
    X_test = df_test.drop(["id", "genre_class", "row_number"], axis=1)
    y_test = df_test["genre_class"]

    # Predict
    knn = get_model_knn()
    distances, indices = knn.kneighbors(X_test, n_neighbors=n_neighbors)

    # Lookup song details
    recommended = pd.DataFrame({"row_number": indices[0], "distances": distances[0]})
    sql = """
        select * from features
        where row_number in (select row_number from recommended)
    """
    details = con.query(sql).df()
    merged = pd.merge(recommended, details, left_on="row_number", right_on="row_number")
    
    merged = merged.filter(
        ["track_name", "artist_name", "album_name", "genres", "genre_class", "preview_url", "track_href"
          "row_number", "distances",
         "acousticness", "danceability", "energy", "instrumentalness",
         "liveness", "loudness", "speechiness", "tempo", "valence"]
    )
    return merged


In [197]:
make_recommendations(con, track_id="5owJQbueZftSai4CuVlR9J")


,track_name,artist_name,album_name,genres,genre_class,preview_url,distances,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,Day 'N' Nite - Radio Edit,Kid Cudi,Day 'N' Nite,fidget house,house,https://p.scdn.co/mp3-preview/3e575bca9959101a1ca1585315cd55186d2d2141?cid=b3cdb16d0df2409abf6a8...,0.000000,0.1230,0.767,0.722,0.003270,0.1180,-5.562,0.0601,130.023,0.962
1,עובר ושווא,The Backyard,החצר האחורית2,israeli pop,pop,https://p.scdn.co/mp3-preview/7849ba7a3a7d7f7bc896aa0885351caab0c19bf2?cid=b3cdb16d0df2409abf6a8...,0.046462,0.1140,0.793,0.736,0.000000,0.1340,-4.505,0.0406,127.018,0.953
2,Loco,Pesado,Todo tuyo,Other,Other,https://p.scdn.co/mp3-preview/d1d4df3a15f2fb2aa661366a7c7430fba522621d?cid=b3cdb16d0df2409abf6a8...,0.067277,0.1120,0.771,0.710,0.000000,0.0721,-7.036,0.0303,136.119,0.962
3,Train in Vain - Remastered,The Clash,London Calling,Other,Other,https://p.scdn.co/mp3-preview/06b9fadab4dd93988aeb5816fe972bf70777ecca?cid=b3cdb16d0df2409abf6a8...,0.070286,0.0922,0.761,0.722,0.000366,0.1320,-8.712,0.0428,122.764,0.966
4,Le Cae Aquí,Freciso,Le Cae Aquí,NoGenre,Other,https://p.scdn.co/mp3-preview/79a3bd563803555ec15ed75d4617b69b6d128057?cid=b3cdb16d0df2409abf6a8...,0.080083,0.1260,0.792,0.729,0.000016,0.0795,-3.122,0.0430,133.063,0.915
5,Sing a Song,"Earth, Wind & Fire",Gratitude,Other,Other,https://p.scdn.co/mp3-preview/025ad33fb8456f8f55ad0c9d0e3a5cb293340844?cid=b3cdb16d0df2409abf6a8...,0.080115,0.1570,0.774,0.698,0.000270,0.0684,-7.500,0.0481,122.325,0.969
6,Be OK,Ingrid Michaelson,Be OK,Other,Other,https://p.scdn.co/mp3-preview/7d4e88030abe29890a86c9a43c3485bbe75eed4d?cid=b3cdb16d0df2409abf6a8...,0.080116,0.1300,0.821,0.773,0.000000,0.1160,-6.741,0.0464,133.053,0.954
7,Carry Go Bring Home,Justin Hinds & The Dominoes,Sinners,rock steady,rock,https://p.scdn.co/mp3-preview/9a4bb8155e1653bb4c4ed63aa3d874199dc57a62?cid=b3cdb16d0df2409abf6a8...,0.082889,0.0655,0.793,0.742,0.000000,0.1420,-7.123,0.0370,124.101,0.971
8,Chu pas là,David Marin,Chu pas là - Single,Other,Other,https://p.scdn.co/mp3-preview/0c235cd8fe251f11cf156c63cb6474fa88d147a3?cid=b3cdb16d0df2409abf6a8...,0.083450,0.0886,0.807,0.676,0.001900,0.1330,-7.263,0.0299,128.139,0.967
9,Il ballo di Simone,Various Artists,"Quei mitici anni, Vol.1",classic italian pop,pop,https://p.scdn.co/mp3-preview/c4f6281398200a565fb90b2511f36171af4aa5fd?cid=b3cdb16d0df2409abf6a8...,0.083499,0.0827,0.782,0.760,0.000000,0.0802,-7.257,0.0303,135.022,0.973


## Clean Up

In [82]:
con.close()